In [ ]:
%matplotlib inline

In [1]:
import serial, threading, sys, time, os, datetime, warnings, numpy as np, matplotlib.pyplot as plt
from IPython.display import clear_output
from timeit import default_timer as timer
sys.path.append('/home/pi/Desktop/ADRNX')
#import SIM900, SIM921, SIM922, SIM970
from SRS import SIM900, SIM921, SIM922, SIM970

In [2]:
mf = SIM900.SIM900('/home/pi/Desktop/ADRNX/SRS/config.yaml',debug=True)
mf.init_mainframe()
s921 = SIM921.SIM921(mf,'s921',debug=True);
s922 = SIM922.SIM922(mf,'s922',debug=True);
s970 = SIM970.SIM970(mf,'s970',debug=True);
mf.register_and_init_mods([s921, s922, s970])
mf.start_comm_threads()
time.sleep(1)
try:
    print mf.excq.get_nowait()
    print "OOPS!"
except:
    print "No exceptions!"
    print(sys.exc_info())
    pass 

SIM mainframe device: /dev/ADR_sim900
Response to IDN: Stanford_Research_Systems,SIM900,s/n130132,ver3.6
Registering mods: [<SRS.SIM921.SIM921 instance at 0x6fdc8fd0>, <SRS.SIM922.SIM922 instance at 0x6fdc8f30>, <SRS.SIM970.SIM970 instance at 0x6fd501c0>]
SIM 921 registered!
SIM 922 registered!
SIM 970 registered!
Mod nums: [1, 6, 7]
Output loop started
Input loop startedInput read thread is up!

No exceptions!
(<class 'Queue.Empty'>, Empty(), <traceback object at 0x6fd50350>)


In [ ]:
print str(bin(int(mf.process_cmd_with_resp('*STB?'))))
print mf.process_cmd_with_resp('LEXE?')
print mf.process_cmd_with_resp('LCME?')
print mf.process_cmd_with_resp('MSGL?')
print mf.process_cmd_with_resp('*ESR?')
print mf.process_cmd_with_resp('*SRE?')
print mf.process_cmd_with_resp('*ESE?')

In [ ]:
print mf.process_cmd_with_resp('*IDN?')
print mf.process_cmd_with_resp('ENSS?')
print mf.process_cmd_with_resp('*PSC?')

In [3]:
s922.get_single_value()

OQADD (t:0): |SNDT 6,"TVAL? 1,1"\n|
OQSEND (t:0): |SNDT 6,"TVAL? 1,1"\n|
Read chars: |'MSG 6,#2125.0'|
Read chars: |'00000E+02\n'|
Resp split result: 
['MSG 6,#2125.000000E+02', '']
Read single line: 'MSG 6,#2125.000000E+02'
['MSG 6', '#2125.000000E+02'] | 6 | 2 | 12 | 'MSG 6,#2125.000000E+02' | '5.000000E+02' |
GOT MSG -> S6:'5.000000E+02'
Got value: |500.0|18:17:44.086498|
'datetime.datetime' object has no attribute 'timestamp'


AttributeError: 'datetime.datetime' object has no attribute 'timestamp'

In [ ]:
with warnings.catch_warnings():
    #warnings.simplefilter('error')
    for i in range(1,2):
        mf.process_cmd_no_resp('SEND 6,"TVAL? 0,1\n"')
        time.sleep(3)
        if i%5 == 0:
            clear_output()
        try:
            print s922.excq.get_nowait()
            warnings.warn("EXCEPTION IN THREAD")
        except:
            #print "No exceptions!"
            pass 
mf.clear_virtual_queues()

In [ ]:
mf.debug = True
s922.debug = True
try:
    with warnings.catch_warnings():
        warnings.simplefilter('error')
        s922.start_data_stream(0)
        for i in range(1,20):        
            #time.sleep(1)
            #print(s922.newdataevent.isSet())
            try:
                print s922.excq.get_nowait()
                warnings.warn("EXCEPTION IN THREAD")
            except:
                #print "No exceptions!"
                pass 
            s922.newdataevent.wait(5)
            try:
                print s922.excq.get_nowait()
                warnings.warn("EXCEPTION IN THREAD")
            except:
                #print "No exceptions!"
                pass 
            if s922.newdataevent.isSet():
                s922.newdataevent.clear()
            else:
                s922.stop_data_stream()
                break
            #print(i)
            if i%5 == 0:
                clear_output()
except Exception as e:
    print(e)
finally:
    s922.stop_data_stream()

In [ ]:
mf.debug = True
s921.debug = True
s921.
try:
    with warnings.catch_warnings():
        warnings.simplefilter('error')
        s921.start_data_stream()
        for i in range(1,20):        
            #time.sleep(1)
            #print(s922.newdataevent.isSet())
            s921.newdataevent.wait(5)
            try:
                print s921.excq.get_nowait()
                warnings.warn("EXCEPTION IN THREAD")
            except:
                #print "No exceptions!"
                pass 
            if s921.newdataevent.isSet():
                s921.newdataevent.clear()
            else:
                s921.stop_data_stream()
                break
            #print(i)
            if i%5 == 0:
                clear_output()
except Exception as e:
    print(e)
finally:
    s921.stop_data_stream()

In [ ]:
try:
    with warnings.catch_warnings():
        warnings.simplefilter('error')
        s921.start_data_stream()
        s922.start_data_stream()        
        for i in range(1,20):        
            time.sleep(1)
            s921.await_next_event(tm=5,clear_before=True,clear_after=False)
            s922.await_next_event(tm=5,clear_before=True,clear_after=False)
            try:
                print(s921.excq.get_nowait(),s922.excq.get_nowait())
                warnings.warn("EXCEPTION IN THREADS")
            except:
                #print "No exceptions!"
                pass 
            if not s921.newdataevent.isSet():            
                s921.stop_data_stream(); s922.stop_data_stream()                
                break
            if not s922.newdataevent.isSet():
                s921.stop_data_stream(); s922.stop_data_stream()
                break
            if i%4 == 0:
                clear_output()
except Exception as e:
    print(e)
finally:
    s921.stop_data_stream(); s922.stop_data_stream()

In [ ]:
s970.stop_data_stream()

In [ ]:
print mf.process_cmd_with_resp('SEND 6,"TERM 0\n"')

In [ ]:
print mf.process_cmd_with_resp('SEND 1,"TVAL? 1\n"')

In [ ]:
print mf.process_cmd_with_resp('SEND 1,"*IDN?\n"')

In [ ]:
print mf.process_cmd_with_resp('SEND 6,"SOUT\n"')

In [ ]:
print mf.process_cmd_with_resp('TERM D,LF\n')
print mf.process_cmd_with_resp('MSGL 128\n')

In [ ]:
mf.get_serial_object().write('SRST 6\n')
mf.get_serial_object().write('AINP? 6\n')
mf.get_serial_object().write('AOUT? 6\n')

In [ ]:
mf.get_serial_object().write('SRST 1\n')

In [ ]:
mf.get_serial_object().write('RPER 66\n')

In [ ]:
mf.get_serial_object().write('*IDN?\n')

In [ ]:
mf.get_serial_object().read(1024)

In [ ]:
mfc.debug = True
s922.dbg = True
s922.start_data_stream(0)
i = 0
try:
    while timer() - s < (24*60*60):
        s922.newdataevent.clear()
        s922.newdataevent.wait(5)
        s922.newdataevent.clear()
        temp = s922.lastvalues
        tm = s922.lastdatatimes
                   
        sys.stdout.flush()
        sys.stderr.flush()
        try:
            print s922.excq.get_nowait()
            print "OOPS!"
        except:
            #print "No exceptions!"
            pass                
        #print("|{:5d}|{:s}|{:8f}".format(i,temp,tm[0]))
        print('{} {:10f} {:f} {:f} {:f} {}'.format(i,temp[0],temp[1],temp[2],temp[3],tm[0]))
        #fl.write('{} {} {}'.format(i,temp,tm[0]))
        temps.append(list(temp));
        times.append(list(tm));
        i+=1
        time.sleep(5)
except Exception as e:
    print e
finally:
    s922.stop_data_stream()
    s922.flush_queue()
    fl.close()

In [ ]:
mfc.stop_comm_threads()